In [18]:
import pandas as pd
import io, gzip
from codecs import decode
import pysrt
import datetime as dt
import base64
import re
from tqdm import tqdm
from os import listdir
import pdb
import dill
from datetime import datetime
import numpy as np
import sys
from xmlrpc.client import ServerProxy
import os as os
import os.path
import json
import time


In [19]:
with open('credentials.pkd','rb') as fp:
    credentials = dill.load(fp)
USER_AGENT=credentials['user']
USERNAME=credentials['user']
PASS=credentials['passw']

In [20]:
OPENSUBTITLES_SERVER = 'http://api.opensubtitles.org/xml-rpc'
LANGUAGE = 'en'

xmlrpc = ServerProxy(OPENSUBTITLES_SERVER,allow_none=True)
data = xmlrpc.LogIn(USERNAME, PASS, LANGUAGE, USER_AGENT)
status=data.get('status')
if status != '200 OK':
    print('Could not log in. Something is wrong... ')
    print("data.get('status')")
    token = None
else:
    token = data.get('token')
    print('Logged in. Acquired token = {}'.format(token))
def server_stat():
    print('------------------Current Stats-----------------')
    for key,item in xmlrpc.ServerInfo(token)['download_limits'].items():
        print('{}: {}'.format(key,item))
    return
def get_down_quota():
    return xmlrpc.ServerInfo(token)['download_limits']['client_download_quota']

Logged in. Acquired token = IoHsDotljUwxIlSEwUDXWdgVq62


In [21]:
server_stat()

------------------Current Stats-----------------
global_24h_download_limit: 200
client_ip: 71.183.36.65
limit_check_by: user_id
user_id: 5937493
client_24h_download_count: 0
client_download_quota: 1000
client_24h_download_limit: 1000


In [28]:
data_path=os.getcwd()+'\\website\\flask_files\\model_db\\'
subs_dir=os.getcwd()+'\\subs_storage\\'
import pickle
def save_obj(obj, name):
    with open(data_path + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, protocol=2)
        
def load_obj(name):
    with open(data_path + name + '.pkd', 'rb') as fp:
        return pickle.load(fp)

In [25]:
movie_database = load_obj('movie_DB_1_7_2018')

# Step 1 - Subtitle_ID's
Using out database of IMDB_ID's we seek 3 Subtitle_ID's in order of quality. Best one is first. We will then attempt download of the best, and if that doesnt work, the next one and so on... We do the search in batches of 500 whichis the maximum allowed.


In [26]:
def filter_one(sr_one):
    if not sr_one:
        return ()
    else:
        try:
            sr=[(item['SubDownloadsCnt'],item['SubRating'],item['IDSubtitleFile'],item['IDMovieImdb']) for item in sr_one 
             if item['SubFormat']=='srt' if not re.findall('.*[^a-zA-Z]cd[^a-zA-Z]',item['MovieReleaseName'].lower())]
            sr_sort=sorted(sr, key=lambda x: -int(x[0]))
            result=tuple([fill_id(sr_sort[0][3])]+[item[2] for item in sr_sort[0:3]])
            #pdb.set_trace()
            return result #returns a tuple 
        except:
            return ()

def fill_id(string):
    #opensubtitles removes the 'tt' and all leading 0's . this function replaces them.
    while len(string) < 7:
        string = '0' + string
    return 'tt' + string
    
def search_imdbid(list_imdbids):
    #SearchSubtitles is crappy because it also gives you 500 search results
    #useless to search for 500 movies at a time since each one might have ~100 results
    #for safety i will search for 10 at a time ~ 2000 requests ~ 40per 10seconds ~ 10-15 mins maybe 
    search_params=[{'sublanguageid':'eng','imdbid':item.strip('tt')} for item in list_imdbids]
    search_data=xmlrpc.SearchSubtitles(token,search_params)
    sr=[]
    try_num=0
    while sr==[]:
        if search_data['status'] == '200 OK':
            sd=search_data['data'] #a long list of search results by imdb_ids
            uniq_id=list(set([item['IDMovieImdb'] for item in sd]))
            sr=[filter_one([item for item in sd if item['IDMovieImdb'] == idnum]) for idnum in uniq_id] 
            #filter_one takes a list of search results for a single imdbid and returns a tuple (imdbid, subid1,subid2,subid3)
            #the above 3 lines will partition the search results sd into a list of lists where each inner list is for one imdbid
            if sr==[]: #sr will be an empty list if no results are found (it is possible)
                return []         
        else:
            if try_num==3: #gives up after 3 times
                print('Tried 3 times. Giving up.')
                return 
            try_num=try_num+1
            print('Error Code: {} Retrying in 10 seconds.'.format(search_data['status']))    
            time.sleep(10)
    return sr

# Take movie_database-queries opebsubs-returns sub_id list 

In [40]:
#LA=list of imdb_ids to get

In [7]:
t0 = time.time()
sub_ids=[]
size=10
for i in tqdm(range(0,len(LA),size)):
    sub_ids=sub_ids+search_imdbid(LA[i:i+size])
print('\nElapsed {} seconds.'.format(time.time()-t0))    

NameError: name 'LA' is not defined

Clean the subs by removing all empty tuples which exist because some subs cannot be found. 
**And write to file.**

In [267]:
sub_id_clean=[item for item in sub_ids if item!=()]
with open('sub_id_temp.pkd', 'wb') as fp:
    dill.dump(sub_id_clean, fp)

In [29]:
sub_id_clean = load_obj('sub_id_clean')
print('Number of sub_ids {}'.format(len(sub_id_clean)))

Number of sub_ids 16893


# Misc Functions

In [30]:
def get_title(imdb_id):
    try:
        ind=[item['imdb_id'] for item in movie_database].index(imdb_id)
        return movie_database[ind]['title'] 
    except:
        return 'Not Found.'

# Downloading the subs

In [31]:
def down_sub(sub_id):
    downsub=xmlrpc.DownloadSubtitles(token, [sub_id])
    if downsub['status'] != '200 OK':
        print('Error Code: {}'.format(downsub['status']))
        server_stat()
        sys.exit('Leaving this world now.')
    return downsub

def decode_sub(enc_subfile):
    dec_data=decode(bytearray(enc_subfile['data'][0]['data'].encode("utf-8")),'base64')
    try:
        sub_txt=gzip.GzipFile(fileobj=io.BytesIO(dec_data)).read().decode("utf-8")       
    except:
        sub_txt=gzip.GzipFile(fileobj=io.BytesIO(dec_data)).read().decode("ISO-8859-1")   
    return sub_txt

def write_to_file(file_name, file_content):
    #writes file_content into an srt file in the subs_storage subfolder of current path
    #if exists prints message
    #if os.path.isfile(os.getcwd()+'\\subs_storage\\'+ file_name + '.srt'):
    #    print('Sub '+ file_name + ' exists already.')
    #else:
    with open(os.getcwd()+'\\subs_storage\\'+ file_name +'.srt','wb') as fp:
        fp.write(file_content.encode('utf-8'))

# THIS PART ACTIVELY DOWNLOADS
Checks to see what *.srt files exist in the folder. 
1. If they are there nothing happens.
2. If they are not there then it uses sub_id_clean file to get the best sub listing. 

<img src="http://clipartix.com/wp-content/uploads/2017/06/Stop-sign-clip-art-microsoft-free-clipart-images-2.png" alt="Le bag of words" title="The bag of words" />

**RUNNING THE NEXT CELL WILL CAUSE DOWNLOADING TO OCCUR. YOU ONLY HAVE 1000 PER DAY.**
**MAKE SURE EVERYTHING IS GOOD BEFORE RUNINNG**

In [32]:
len(sub_id_clean)

16893

In [34]:
for item in sub_id_clean:
    if os.path.isfile(os.getcwd()+'\\subs_storage\\'+ item[0] + '.srt'):
        continue
        #print('SUB_ID:{:<10} IMDB_ID:{} exists ---- SKIPPING FILE Title: {}'.format(item[1],item[0],get_title(item[0])))
    else:
        print('SUB_ID:{:<10} IMDB_ID:{} downloading ------------- Title: {}'.format(item[1],item[0],get_title(item[0])))
        write_to_file(item[0],decode_sub(down_sub(item[1])))

SUB_ID:1955007381 IMDB_ID:tt4660980 downloading ------------- Title: The Bad Education Movie
SUB_ID:1952007158 IMDB_ID:tt0103805 downloading ------------- Title: Beyond The Law
SUB_ID:255284     IMDB_ID:tt0064117 downloading ------------- Title: Cactus Flower
SUB_ID:1952169634 IMDB_ID:tt0062626 downloading ------------- Title: 5 Card Stud
SUB_ID:1955585094 IMDB_ID:tt0116016 downloading ------------- Title: Danger Zone
SUB_ID:313176     IMDB_ID:tt0067055 downloading ------------- Title: Equinox
SUB_ID:1954128243 IMDB_ID:tt1638364 downloading ------------- Title: The Armstrong Lie
SUB_ID:261021     IMDB_ID:tt0116493 downloading ------------- Title: Harriet the Spy
SUB_ID:1953021384 IMDB_ID:tt1606390 downloading ------------- Title: All Things Fall Apart
SUB_ID:1951964051 IMDB_ID:tt0041996 downloading ------------- Title: Twelve O'Clock High
SUB_ID:1953420350 IMDB_ID:tt0272018 downloading ------------- Title: Call Me Claus
SUB_ID:1954271209 IMDB_ID:tt1993391 downloading ------------- Titl

SUB_ID:1951855186 IMDB_ID:tt0052993 downloading ------------- Title: Last Train from Gun Hill
SUB_ID:1952066711 IMDB_ID:tt0102395 downloading ------------- Title: Mannequin Two: On the Move
SUB_ID:1954474976 IMDB_ID:tt0093744 downloading ------------- Title: The New Adventures of Pippi Longstocking
SUB_ID:1954482090 IMDB_ID:tt3005242 downloading ------------- Title: Cantinflas
SUB_ID:1952611821 IMDB_ID:tt1424003 downloading ------------- Title: The Infidel
SUB_ID:1954706261 IMDB_ID:tt3905756 downloading ------------- Title: Her Infidelity
SUB_ID:1951921328 IMDB_ID:tt0407658 downloading ------------- Title: The Chronicles of Riddick: Dark Fury
SUB_ID:257851     IMDB_ID:tt0066518 downloading ------------- Title: The Vampire Lovers
SUB_ID:1951931220 IMDB_ID:tt1041753 downloading ------------- Title: Harold
SUB_ID:1954848657 IMDB_ID:tt3200980 downloading ------------- Title: Where Hope Grows
SUB_ID:1951718788 IMDB_ID:tt0074906 downloading ------------- Title: The Missouri Breaks
SUB_ID:195

SUB_ID:1954992266 IMDB_ID:tt3727690 downloading ------------- Title: 2 Guns: Zero Tolerance
SUB_ID:1952172201 IMDB_ID:tt0018528 downloading ------------- Title: The Unknown
SUB_ID:1951667896 IMDB_ID:tt0409043 downloading ------------- Title: Grilled
SUB_ID:1952979993 IMDB_ID:tt0100409 downloading ------------- Title: The Prince and the Pauper
SUB_ID:1952015866 IMDB_ID:tt1236370 downloading ------------- Title: Legendary Assassin
SUB_ID:311970     IMDB_ID:tt0090570 downloading ------------- Title: 84 Charing Cross Road
SUB_ID:1954223955 IMDB_ID:tt0086355 downloading ------------- Title: Star 80
SUB_ID:1954344096 IMDB_ID:tt0073906 downloading ------------- Title: The Wind and the Lion
SUB_ID:1954632137 IMDB_ID:tt0060121 downloading ------------- Title: Arabesque
SUB_ID:1953579888 IMDB_ID:tt2179226 downloading ------------- Title: Absolute Deception
SUB_ID:1951896802 IMDB_ID:tt0111157 downloading ------------- Title: Sharpe's Enemy
SUB_ID:1951614910 IMDB_ID:tt0065836 downloading ---------

SUB_ID:1954564004 IMDB_ID:tt2295564 downloading ------------- Title: McCanick
SUB_ID:257092     IMDB_ID:tt0067805 downloading ------------- Title: Sunday Bloody Sunday
SUB_ID:1955614841 IMDB_ID:tt0133985 downloading ------------- Title: A Murder of Crows
SUB_ID:224456     IMDB_ID:tt0058283 downloading ------------- Title: Lady in a Cage
SUB_ID:1954853510 IMDB_ID:tt3111486 downloading ------------- Title: American Beach House
SUB_ID:1952280307 IMDB_ID:tt1290400 downloading ------------- Title: Double Identity
SUB_ID:1952910786 IMDB_ID:tt1787759 downloading ------------- Title: Madea's Big Happy Family
SUB_ID:1952293248 IMDB_ID:tt0058580 downloading ------------- Title: Sex and the Single Girl
SUB_ID:1955429727 IMDB_ID:tt3568002 downloading ------------- Title: The Witness
SUB_ID:127626     IMDB_ID:tt0087428 downloading ------------- Title: The Hotel New Hampshire
SUB_ID:1952142124 IMDB_ID:tt0100151 downloading ------------- Title: Mindwalk
SUB_ID:1951697141 IMDB_ID:tt0077714 downloading

SUB_ID:1954857630 IMDB_ID:tt1772261 downloading ------------- Title: The Curse of Downers Grove
SUB_ID:1954946722 IMDB_ID:tt4247618 downloading ------------- Title: The Dresser
SUB_ID:1955370588 IMDB_ID:tt4846318 downloading ------------- Title: Into the Inferno
SUB_ID:1952905325 IMDB_ID:tt0047878 downloading ------------- Title: The Big Combo
SUB_ID:1952741990 IMDB_ID:tt1711366 downloading ------------- Title: Dead Space: Aftermath
SUB_ID:1954372862 IMDB_ID:tt3461828 downloading ------------- Title: Cat Run 2
SUB_ID:1951983656 IMDB_ID:tt0494277 downloading ------------- Title: Strictly Sexual
SUB_ID:1953486757 IMDB_ID:tt0162360 downloading ------------- Title: Happy, Texas
SUB_ID:1955489584 IMDB_ID:tt5834800 downloading ------------- Title: Devil in the Dark
SUB_ID:1953692334 IMDB_ID:tt0080436 downloading ------------- Title: The Big Brawl
SUB_ID:1954945836 IMDB_ID:tt0080603 downloading ------------- Title: Death Ship
SUB_ID:1952299331 IMDB_ID:tt0091814 downloading ------------- Title

SUB_ID:1954982634 IMDB_ID:tt0167758 downloading ------------- Title: Alice Through the Looking Glass
SUB_ID:1955134704 IMDB_ID:tt5529680 downloading ------------- Title: Darth Maul: Apprentice
SUB_ID:1952207623 IMDB_ID:tt1405412 downloading ------------- Title: Hardwired
SUB_ID:1955671300 IMDB_ID:tt0234516 downloading ------------- Title: The President's Man
SUB_ID:1952835182 IMDB_ID:tt0039220 downloading ------------- Title: Brighton Rock
SUB_ID:238958     IMDB_ID:tt0327643 downloading ------------- Title: Dirty Love
SUB_ID:1951695796 IMDB_ID:tt0058530 downloading ------------- Title: Robinson Crusoe on Mars
SUB_ID:1954610137 IMDB_ID:tt4215766 downloading ------------- Title: Scooby-Doo! Moon Monster Madness
SUB_ID:1954140591 IMDB_ID:tt3277552 downloading ------------- Title: Aningaaq
SUB_ID:1951823951 IMDB_ID:tt0258273 downloading ------------- Title: Lovely & Amazing
SUB_ID:1952830742 IMDB_ID:tt1559033 downloading ------------- Title: Prowl
SUB_ID:1954676131 IMDB_ID:tt2194826 downlo

SUB_ID:1952908294 IMDB_ID:tt0043455 downloading ------------- Title: David and Bathsheba
SUB_ID:1954886824 IMDB_ID:tt2392810 downloading ------------- Title: Queen & Country
SUB_ID:1953908526 IMDB_ID:tt0098090 downloading ------------- Title: The Phantom of the Opera
SUB_ID:1955742554 IMDB_ID:tt6719524 downloading ------------- Title: 5 Headed Shark Attack
SUB_ID:1952316559 IMDB_ID:tt0113360 downloading ------------- Title: The Hunted
SUB_ID:1952865569 IMDB_ID:tt1283887 downloading ------------- Title: Burning Palms
SUB_ID:1955011985 IMDB_ID:tt0093974 downloading ------------- Title: Silent Night, Deadly Night II
SUB_ID:181573     IMDB_ID:tt0102443 downloading ------------- Title: A Midnight Clear
SUB_ID:1955049944 IMDB_ID:tt5039822 downloading ------------- Title: Addicted to Sexting
SUB_ID:1954386261 IMDB_ID:tt0077372 downloading ------------- Title: Corvette Summer
SUB_ID:1952211217 IMDB_ID:tt1210801 downloading ------------- Title: Command Performance
SUB_ID:1954930572 IMDB_ID:tt33

SystemExit: Leaving this world now.

C:\Users\Adi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Parsing SRT Files

In [135]:
subs_dir=os.getcwd()+'\\subs_storage\\'
def read_srt(imdb_id):
    if os.path.isfile(subs_dir+imdb_id + '.srt'):
        with open(subs_dir+imdb_id+'.srt','rb') as fp:
            test=fp.read().decode('utf-8')
        return test
    else:
        pass

def fparse1(string):
    if string:
        #string=re.sub(r'<i>','',string)    
        #string=re.sub(r'</i>','',string)
        string=re.sub('<[^>]*.','',string) #kills anything inside < CACA >
        string=re.sub(r'\ufeff','',string)
        #string=re.sub('^[^(1)]*','',string) #removes stuff before the 1
        string=re.sub('^[^\a]*?1\r\n00:','1\r\n00:',string) #removes stuff before the 1 betterly
        return string
    else:
        pass

def srt_conv(subs):
    if subs:
        starttimes=[]
        endtimes=[]
        words=[]
        indexes=[]
        #"-->" in line
        subs=io.StringIO(subs)
        prevLineWasWords =False
        prevLineWasTime =False
        for line in subs:
            line=line.strip()
            if "-->" in line:
                starttimes.append(line[:8])
                endtimes.append(line[17:25])
                prevLineWasTime =True
            elif line.strip().isdigit() and not prevLineWasTime:
                indexes.append(int(line))
                prevLineWasTime=False
            else:
            #elif len(line) > 1:
                prevLineWasTime = False
                if prevLineWasWords:
                        words[len(words)-1] = words[len(words)-1] + " " + line
                else:
                        words.append(line)
                prevLineWasWords = True
                continue
            prevLineWasWords = False  
        
        #trim because sometimes a file will end without a last tine.
        trim_by=min([len(starttimes),len(endtimes),len(words)])
        starttimes=starttimes[0:trim_by]
        endtimes=endtimes[0:trim_by]
        words=words[0:trim_by]
        
        return starttimes, endtimes, words, indexes
    else:
        pass
    
def watch_output(clean_srt):
    subs=io.StringIO(clean_srt)
    print('=== is number, ~~~ is text, --- is time index. \n')
    prevLineWasTime=False
    for line in subs:
        if "-->" in line:
            prevLineWasTime=True
            print('----'+line)
        elif line.strip().isdigit() and not prevLineWasTime:
            prevLineWasTime=False
            print('====='+line)
        else:
        #elif len(line)>1:
            prevLineWasTime=False
            print('~~~~'+line)
    return

def s_time(string):
    try:
        a=datetime.strptime(string, '%H:%M:%S')
        return a
    except:
        pass
    

def make_df(imdb_id):
    srt1=fparse1(read_srt(imdb_id))
    if srt1:
        a,b,c,_=srt_conv(srt1)

        imdb_idi=[imdb_id]
        linfo=['StartTime','EndTime','Line']
        index=pd.MultiIndex.from_product([imdb_idi,linfo],names=['IDs','LData'])
        data_sub=pd.DataFrame(columns=index)


        s1 = pd.Series([s_time(item) for item in a])
        s2 = pd.Series([s_time(item) for item in b])
        s3 = pd.Series(c)
        data_sub.loc[:,(imdb_id,'StartTime')] =s1.values
        data_sub.loc[:,(imdb_id,'EndTime')] = s2.values
        data_sub.loc[:,(imdb_id,'Line')] = s3.values

        return data_sub
    else:
        pass
    
    
def make_tup(imdb_id):
    srt1=fparse1(read_srt(imdb_id))
    if srt1:
        a,b,c,_=srt_conv(srt1)

        s1 = [s_time(item) for item in a]
        s2 = [s_time(item) for item in b]
        s3 = c

        return (s1,s2,s3)
    else:
        pass    

def append_pd(df1,df):
    tes=pd.concat([df1,df], axis=1)
    tes=tes.fillna('')
    return tes

In [133]:
aid=[item.split('.')[0] for item in listdir(subs_dir) if 'tt' in item]

In [138]:
t=time.time()
full_data={}
for i in range(len(aid)):
    full_data[aid[i]]=make_tup(aid[i])
    if i%100==0:
        print('No. {} - adding {} - elapsed {} sec.'.format(i,aid[i],time.time()-t))
t1=time.time()
print('elapsed time {} seconds'.format(t1-t))

No. 0 - adding tt0002381 - elapsed 0.058014869689941406 sec.
No. 100 - adding tt0029583 - elapsed 2.318572759628296 sec.
No. 200 - adding tt0037522 - elapsed 6.224541425704956 sec.
No. 300 - adding tt0044081 - elapsed 10.13450312614441 sec.
No. 400 - adding tt0049949 - elapsed 13.515328407287598 sec.
No. 500 - adding tt0056193 - elapsed 17.217242002487183 sec.
No. 600 - adding tt0062190 - elapsed 20.68910241127014 sec.
No. 700 - adding tt0067328 - elapsed 24.059925317764282 sec.
No. 800 - adding tt0072431 - elapsed 27.283752918243408 sec.
No. 900 - adding tt0078736 - elapsed 31.405113220214844 sec.
No. 1000 - adding tt0082817 - elapsed 35.27142262458801 sec.
No. 1100 - adding tt0086837 - elapsed 39.01010704040527 sec.
No. 1200 - adding tt0090297 - elapsed 42.65796709060669 sec.
No. 1300 - adding tt0093223 - elapsed 46.41611313819885 sec.
No. 1400 - adding tt0096328 - elapsed 50.37458324432373 sec.
No. 1500 - adding tt0099557 - elapsed 54.393736600875854 sec.
No. 1600 - adding tt0103010